# Working with timezones

Timestamp data can be timezone-naíve or timezone-aware. Using timestamped data with *staircase* has been a feature since the inception of the package, however the ability to work with timezone-aware date was only introduced in v1.6.0.

For many time-based applications of staircase it may suffice to ignore the concept of timezones and work with timezone-naíve data - an attractive option as working with timezones, and converting between them, can be tricky.  However many countries observe [Daylight Savings Time](https://en.wikipedia.org/wiki/Daylight_saving_time) which results in one day of the year having 23 hours, and another having 25 hours.  Now if you are computing some daily metric and do not take this into account then the calculations on those days will be incorrect, however the consequences, and indeed the calculated result, could well be immaterial.  However, that would be a lazy excuse for not implementing timezone support in staircase, and indeed for some applications the use of timezone-aware timestamps may be critical.

Given the sheer number of packages available for Python it may be of no surprise that there are several for dealing with timezones however there is one which is clearly the de facto standard: [pytz](http://pytz.sourceforge.net/).  We will demonstrate the use of Pytz in this article, however staircase supports any timezone package that [pandas](https://pandas.pydata.org/) supports.

In [ ]:
import pandas as pd
import staircase as sc
import matplotlib.pyplot as plt
import pytz

Timezone-aware timestamps can be created directly in pandas:

In [ ]:
timezone = pytz.timezone('Australia/Sydney')
ts_aware = pd.Timestamp('2020', tz=timezone)
ts_aware

but they can also be created by *localizing* existing timezone-naive timestamps.

In [ ]:
ts_naive = pd.Timestamp('2020')
ts_aware = ts_naive.tz_localize(timezone)
ts_aware

Notice that the offset for Jan 1st 2020 is +11 hours.  This means that the time in Sydney, Australia, at that precise moment is 11 hours ahead of [Coordinated Universal Time](https://en.wikipedia.org/wiki/Coordinated_Universal_Time)  (UTC).  UTC is a successor to Greenwich Mean Time and is now the standard by which clocks and time are regulated around the world.  If we create a timezone-aware timestamp for the 1st of May 2020, then we get a different offset of +10 hours:

In [ ]:
pd.Timestamp('2020-5-1', tz=timezone)

This is because on the 5th of April 2020, Sydney (and the surrounding areas which observe the same timezone), wound clocks back at 3am to once again read 2am, in observance of Daylight Savings Time.  This means that the 5th of April was 25 hours long:

In [ ]:
(pd.Timestamp('2020-4-6', tz=timezone) - pd.Timestamp('2020-4-5', tz=timezone))/pd.Timedelta('1h')

The fact that the 5th of April saw two occurences of 2am (and every time between 2am and 3am) is one of the reasons why timezones can be tricky - particularly if there is timezone-naíve timestamps which need to be made timezone-aware.

Daylight savings in Sydney resumes on the 4th of October 2020, where at 2am the clocks are wound forward an hour.  This means that 2:30am on the 4th of October 2020, for example, is not a time that exists in that timezone.  If we try to localize the corresponding timestamp we get an exception:

In [ ]:
try:
    pd.Timestamp('2020-10-4 2:30').tz_localize(timezone)
except:
    import sys
    error = sys.exc_info()[1]
    print(f"{error.__class__.__name__}: {error}")

We can also localize pandas Series of timestamps.  For example, if we have the following timezone-naíve data:

In [ ]:
data = pd.read_csv(r"../examples/data/asset_use.csv", parse_dates=['start', 'end'], dayfirst=True)
series_of_timestamps = data.start

In [ ]:
series_of_timestamps

then we can convert all timestamps to be timezone-aware by first using the [dt accessor](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) and then calling the *tz_localize* method. 

In [ ]:
series_of_timestamps_tz = series_of_timestamps.dt.tz_localize(timezone)
series_of_timestamps_tz

As mentioned earlier we can use timezone-aware data with staircase.  In version 1 we were required to declare the timezone when initialising a Stairs object, however this is not necessary in version 2.

In [ ]:
example_tz = sc.Stairs()
example_tz

In the example below we use the first 6 timestamps from the above series to add layers.

In [ ]:
example_tz.layer(series_of_timestamps_tz[:3], series_of_timestamps_tz[3:6])
example_tz.plot(ax=plt.subplots(figsize=(10,3))[1])

Note that once we start using with a timezone-aware data with a Stairs object, then all interactions with that object must be done in the context of that timezone.  A similar requirement is found in binary operations with pandas.Timestamps, such as subtraction or greater-than, which are only permitted if the timestamps are either timezone-naíve or have the same timezone.

For staircase, we require (when using Stairs with dates):

- using a timezone-aware timestamp when using the *Stairs.sample* method
- both operands in binary operations, such as + or >=, must both be timezone-naíve or have the same timezone
- *lower* and *upper* parameters in methods such as *mean*, *integrate*, *describe*, *hist* etc must have the same timezone as the Stairs instance the method is called on

This last requirement in particular may lead to constantly needing to localize timestamps, eg:

In [ ]:
example_tz.clip(
    pd.Timestamp('2020-1-1 1:00', tz=timezone),
    pd.Timestamp('2020-1-1 3:00', tz=timezone),
).mean()

Users of staircase may find it useful to create a function to adds timezones to the timestamps, and use in place of the pd.Timestamp constructor:

In [ ]:
tz_timestamp = lambda t: pd.Timestamp(t, tz=timezone)
example_tz.clip(
    tz_timestamp('2020-1-1 1:00'),
    tz_timestamp('2020-1-1 3:00'),
).mean()

For a further example of working with timezones see [Case study: asset utilisation (with timezones)](../examples/Case Study Asset Utilisation TZ.ipynb).